### Testing protein representations using linear regression

In [1]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2
import os
import pandas as pd 
import numpy as np 
import tqdm 
import glob


import torch
import torch.nn as nn 
import torch.optim as optim 

import torchvision 
import torchvision.transforms as transforms 
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, IterableDataset, DataLoader

from sklearn.linear_model import Lasso

import protrnn as pr

seed = 4
np.random.seed(seed)
torch.manual_seed(seed)


import bokeh
import bokeh_catplot
import holoviews as hv
hv.extension('bokeh')


The purpose of this notebook is to use our previously trained model to predict a property of some input proteins. For this example we'll be using the localization dataset from [*Yang et al.*](https://academic.oup.com/bioinformatics/article/34/15/2642/4951834) from the Arnold Lab at Caltech.

This dataset comes from [a previous paper](https://www.pnas.org/content/pnas/114/13/E2624.full.pdf) from the Arnold Lab, were they used a recombination-based method to generate a library of new Channelrhodopsins. They measured expression, localization and localization efficiency. We'll be using the localization metric, which represents the amount of fluorescence from GFP (in arbitrary units). The way they measure localization is via the [SpyTag/SpyCatcher system ](https://www.sciencedirect.com/science/article/pii/S107455211500246X?via%3Dihub). In brief, they use a small tag (SpyTag) to label the Channelrhodopsins. Those Channelrhodopsins that get to the membrane will find their cognate GFP label with a complementary sequence to the tag (SpyCatcher). In this way, one can measure the amount of proteins that localize to the membrane using fluorescence imaging. 

The task we have at hand is, to train a model, that will predict the amount of localized protein (reported in log(GFP) units) using the vectorized representation from the hidden state of our RNN after passing the protein through the net.  Let's get started! 

### Load pretrained model 

In [2]:
aminoacid_list = [
    'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
    'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'
]

In [3]:
embedding_size= 10
hidden_size = 128
n_aminoacids = len(aminoacid_list) + 1

rnn = pr.RNN_GRU(
    input_size = n_aminoacids, 
    embedding_size = embedding_size, 
    hidden_size = hidden_size, 
    output_size = n_aminoacids
)

In [4]:
path = '../../../thomson_lab/data/random_split/'

In [5]:
ls {path}

aa.csv          dev_filt.csv    rnn_gru.pt      test_filt.csv   train_filt.csv
dev/            dev_filt_.csv   test/           train/


In [6]:
rnn.load_state_dict(torch.load(path + 'rnn_gru.pt'))

<All keys matched successfully>

In [7]:
rnn

RNN_GRU(
  (log_softmax): LogSoftmax()
  (embedding): Embedding(21, 10)
  (gru): GRU(10, 128)
  (decoder): Linear(in_features=128, out_features=21, bias=True)
)

### Load Yang et al enantioselectivity dataset 

In [8]:
path_data = '~/Downloads/kkyang_SI/'

In [9]:
df = pd.read_csv(path_data + 'localization.txt')

In [10]:
df.head()

,name,sequence,log_GFP,is_train,m
0,cschrimson,MSRLVAASWLLALLLCGITSTTTASAHIVMVDAYKPTKSAPAASST...,-4.626936,True,0.0
1,c1c2,MSRRPWLLALALAVALAAGSAGAAHIVMVDAYKPTKSTGSDATVPV...,-5.599110,True,0.0
2,cheriff,MGGAPAPDAHSAPPGNDSAAHIVMVDAYKPTKGGSEYHAPAGYQVN...,-5.715788,True,0.0
3,c1,MSRRPWLLALALAVALAAGSAGAAHIVMVDAYKPTKSTGSDATVPV...,-5.335352,True,32.0
4,c2,MSRRPWLLALALAVALAAGSAGAAHIVMVDAYKPTKSTGSDATVPV...,-4.187052,True,26.0


In [11]:
# Check if columns have nans
df.isna().sum()

name        0
sequence    0
log_GFP     6
is_train    0
m           0
dtype: int64

Let's remove those proteins with NaNs in the dataset. 

In [12]:
df.shape

(254, 5)

In [13]:
df = df.dropna()

In [14]:
df.shape

(248, 5)

First-off, let's look at the distribution of GFP values of the train and test set. 

In [167]:
p= bokeh_catplot.ecdf(df, val = 'log_GFP', cats = ['is_train'])

In [168]:
bokeh.io.show(p)

We can see that the test set is a bit bimodal, but overall, the support seems to be fairly splitted between the datasets. Let's proceed with our analysis. 

In [17]:
df_train = df[df.is_train==True]
df_test = df[df.is_train==False]

In [18]:
print(f'Number of proteins in training set: {len(df_train)}')
print(f'Number of proteins in test set: {len(df_test)}')

Number of proteins in training set: 215
Number of proteins in test set: 33


In [19]:
# Get a sense of the number of mutations. 
df.m.value_counts().head()

10.0    19
17.0     9
25.0     7
6.0      7
39.0     7
Name: m, dtype: int64

### Project sequences to hidden state space

Let's remind ourselves of the output of our function by trying out one sequence. 

In [20]:
# Get first protein of trains set
prot = df_train.sequence.values[0]
# See first 10 aminoacids 
prot[:10]

'MSRLVAASWL'

In [23]:
# Initialize aminoacid vocab
aminoacid_vocab = pr.aminoacid_vocab()

In [24]:
# Transfer function from vocab object
aminoacid_to_index = aminoacid_vocab.aminoacid_to_index

In [25]:
# Get hidden state for first protein in train set 
with torch.no_grad():
    hidden= pr.get_hidden_state(
        prot, rnn, hidden_size, aminoacid_to_index, get_last_hidden=True
)

In [26]:
# Check dimensionality of the output hidden state 
hidden.shape

(128,)

We can see that the hidden state is what we expect, a 128 dim vector. We can proceed run the sequences through our trained RNN. 

In [27]:
# Get arrays of protein sequences. 
seqs_train = df_train.sequence.values
seqs_test = df_test.sequence.values

In [45]:
# Initialize empty arrays for storing hidden states  
hiddens_train = np.zeros(shape = (seqs_train.shape[0], hidden_size))
hiddens_test = np.zeros(shape = (seqs_test.shape[0], hidden_size))

In [47]:
# Project! 
with torch.no_grad():
    
    print('Converting train sequences...')
    for i, prot in tqdm.tqdm(enumerate(seqs_train)): 
    
        hidden = pr.get_hidden_state(
            prot, rnn, hidden_size, aminoacid_to_index, get_last_hidden=True
        )

        # Add hidden state for current protein to array using numpy indexing 
        hiddens_train[i, :] = hidden   
    
    print('Converting test sequences...')
    for i, prot in tqdm.tqdm(enumerate(seqs_test)):
        hidden = pr.get_hidden_state(
            prot, rnn, hidden_size, aminoacid_to_index, get_last_hidden=True
        )
        
        # Add protein to array 
        hiddens_test[i, :] = hidden   

9it [00:00, 84.95it/s]

Converting train sequences...


215it [00:02, 97.33it/s] 
10it [00:00, 95.86it/s]

Converting test sequences...


33it [00:00, 91.85it/s]


In [48]:
hiddens_train.shape

(215, 128)

In [49]:
hiddens_test.shape

(33, 128)

### Train linear model. 

All right. We now have to set our regression task. We will be using the Lasso method. Very briefly, the Lasso regression is the equivalent of linear regression using a Laplace prior, or the so called L1 regularization. The effect of using this method is that it inforces sparsity on the coefficients of our input variables, namely the hidden states. In this sense, *uninformative* dimensions of the hidden state, will be discarded for prediction. Let's go ahead and initialize our model and predict the log(GFP) values for the test set. 

In [50]:
y_train = df_train.log_GFP.values
y_test = df_test.log_GFP.values

In [51]:
# Initialize and train linear model
regression = Lasso(random_state = seed, alpha = 1e-3).fit(hiddens_train, y_train)

/Users/manoloflores/anaconda3/envs/maxwell/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.12594222836418112, tolerance: 0.03265761071871562
  positive)


In [55]:
# Make predicttions for test set 
y_pred = regression.predict(hiddens_test)

In [91]:
n_seqs_test = df_test.shape[0]

In [92]:
# Mean squared error
mse = np.sum((y_pred-y_test)**2)/n_seqs_test

In [93]:
mse

1.406076522387037

In [94]:
# Mean absolute error 
mae = np.sum(np.abs(y_pred-y_test))/n_seqs_test

In [95]:
mae

0.9935075397425209

In [96]:
line = np.linspace(y_pred.min()-1, y_pred.max()+1, 20)

In [97]:
line_plot = hv.Curve((line, line)).opts(color = 'grey')

In [153]:
prediction_plot = hv.Scatter((y_test, y_pred)).opts(
    padding=0.1,
    ylabel="predicted log(GFP)",
    xlabel="real log(GFP)",
    alpha=0.8,
    color="#756bb1",
    size = 7,
    title = 'Localization prediction using Lasso, MAE: %.3f'%mae,
    width = 440
)

In [154]:
# Overlay plots using holoviews "grammar"
prediction_plot*line_plot

:Overlay
   .Scatter.I :Scatter   [x]   (y)
   .Curve.I   :Curve   [x]   (y)

Neat! We can see that the predictions are not too off from the measurements. Well, at least One question that sparks to my mind is to see whether the number of mutations i.e. single aminoacid deviations from the parents make the prediction harder. 

We can add the the predictions to the test dataframe and color the dots using hvplot. 

In [101]:
df_test['predicted_log_gfp'] = y_pred

/Users/manoloflores/anaconda3/envs/maxwell/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [102]:
df_test.head()

,name,sequence,log_GFP,is_train,m,predicted_log_gfp
201,n1_2,MGGAPAPDAHSAPPGNDSAAHIVMVDAYKPTKGGSEYHAPAGYQVN...,-7.327994,False,87.0,-8.570076
202,n2_2,MSRLVAASWLLALLLCGITSTTTASAHIVMVDAYKPTKSAPAASST...,-7.312336,False,55.0,-7.905948
203,n3_2,MGGAPAPDAHSAPPGNDSAAHIVMVDAYKPTKGGSEYHAPAGYQVN...,-7.742237,False,81.0,-8.701904
204,n4_2,MSRRPWLLALALAVALAAGSAGAAHIVMVDAYKPTKSTGSDATVPV...,-5.950239,False,54.0,-5.791737
205,n5_2,MSRLVAASWLLALLLCGITSTTTASAHIVMVDAYKPTKSAPAASST...,-5.012321,False,57.0,-5.904190


In [115]:
df_train.m.mean()

44.44186046511628

In [116]:
df_test.m.mean()

45.96969696969697

In [103]:
import hvplot.pandas

In [129]:
df_test.hvplot.scatter(
    x = 'log_GFP', 
    y = 'predicted_log_gfp', 
    color = 'm',
    width = 500, 
    size = 50, 
    alpha = 0.9,
    xlabel = 'measured log (GFP)',
    ylabel = 'predicted log (GFP)',
    cmap = 'viridis_r', 
    clabel = '# mutations'
)*line_plot

:Overlay
   .Scatter.I :Scatter   [log_GFP]   (predicted_log_gfp,m)
   .Curve.I   :Curve   [x]   (y)

We can see that there is not really a visible trend whereby the highly mutated sequences deviate from the prediction line. If anything, the model makes the worst predictions (underestimates) the proteins with the highest expression. Let's see if this is somehow an hard extrapolation error due to the training set not containing proteins with as much localization as the test set. 

In [159]:
df_train.log_GFP.sort_values(ascending = False).head(10)

222   -4.040097
62    -4.153545
4     -4.187052
218   -4.340050
156   -4.353863
102   -4.558817
167   -4.573850
169   -4.576447
158   -4.585620
67    -4.593912
Name: log_GFP, dtype: float64

In [160]:
df_test.log_GFP.sort_values(ascending = False).head(10)

253   -3.958357
237   -4.088195
252   -4.148785
247   -4.308444
250   -4.418679
208   -4.598022
251   -4.606671
211   -4.700758
205   -5.012321
209   -5.061021
Name: log_GFP, dtype: float64

We can see that the training set does contain a lot of proteins with high GFP  (high localization) and therefore this is not the cause of the higher error in high expressors. Future work could focus on playing with the parameter $\alpha$ that controls the sparsity inducing force of Lasso. 

#### Reproducibility

In [162]:
%load_ext watermark
%watermark -v -m -p  sklearn,torch,numpy,pandas,holoviews,bokeh

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.7.6
IPython 7.16.1

sklearn 0.23.1
torch 1.5.1
numpy 1.19.1
pandas 1.1.0
holoviews 1.13.3
bokeh 2.1.1

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
